# IS-LM model

In [1]:
"1|IMPORT PACKAGES"
import numpy as np               # Package for scientific computing with Python
import matplotlib.pyplot as plt  # Matplotlib is a 2D plotting library
from scipy import linalg
from scipy import optimize
import sympy as sm
from types import SimpleNamespace

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

## Analytical solution
First we define all symbols used in both the simple and the extended IS-LM model

In [4]:
#production function
Y = sm.symbols('Y')
C = sm.symbols('C')
I = sm.symbols('I')
G = sm.symbols('G')
X = sm.symbols('X')
Z = sm.symbols('Z')

#new variables 
Yn = sm.symbols('Y^{new}')

#consumption function
a = sm.symbols('a')
b = sm.symbols('b')
T = sm.symbols('T')

#Investment and the interest rate
I = sm.symbols('I')
I_bar = sm.symbols('I_bar')
d = sm.symbols('d')
i = sm.symbols('i')

#imports
alpha = sm.symbols('alpha')
beta = sm.symbols('beta')

G_bar = sm.symbols('G_bar')
X_bar = sm.symbols('X_bar')

#
c_1 = sm.symbols('c_1')
Ms = sm.symbols('Ms')
P = sm.symbols('P')
c_3 = sm.symbols('c_3')
c_2 = sm.symbols('c_2') 


We turn i_IS equation into a Python solution with the usage of lambdify

In [3]:
#Defining i_IS
"3|DEFINE AND POPULATE THE IS-SCHEDULE"
def i_IS(a, alpha, b, beta, T, I_bar, G_bar, X_bar, d, Y):
    i_IS = ((a-alpha)-(b-beta)*T + I_bar + G_bar + X_bar - (1-b+beta)*Y)/d
    return i_IS

def Y_IS(a, alpha, b, beta, T, I_bar, G_bar, X_bar, d, i):
    Y_IS = ((a-alpha)-(b-beta)*T + I_bar + G_bar + X_bar - d*i)/(1-b+beta)
    return Y_IS

i = i_IS(a, alpha, b, beta, T, I_bar, G_bar, X_bar, d, Y)

y = Y_IS(a, alpha, b, beta, T, I_bar, G_bar, X_bar, d, i)

In [4]:
i_IS_func = sm.lambdify(args = (a, alpha, b, beta, T, I_bar, G_bar, X_bar, d, Y),
                     expr = i)


In [6]:
"2|DEFINE PARAMETERS AND ARRAYS"
# Parameters
Y_value = 100
a_value = 20                 # Autonomous consumption
b_value = 0.2                # Marginal propensity to consume
alpha_value = 5              # Autonomous imports
beta_value  = 0.2            # Marginal propensity to import
T_value = 1                  # Taxes
I_bar_value = 10             # Investment intercept (when i = 0)
G_bar_value = 8              # Government spending
X_bar_value = 2              # Exports (given)
d_value = 5                  # Investment slope wrt to i
# Arrays
Y = np.arange(Y_value)  # Array values of Y

i_IS_ana = i_IS_func(a_value, alpha_value, b_value, beta_value, T_value, I_bar_value, G_bar_value, X_bar_value, d_value, Y_value)

print(f' The analytical solution for i_IS is: {i_IS_ana:.3f}')

 The analytical solution for i_IS is: -13.000


When symbols are defined, we define the equations contained in the model

In [5]:
#Defining the equations as variables
Y_func = C + I + G + (X-Z)
C_func = a + b*(Y-T)
I_func = I_bar - d*i
Z_func = alpha + beta*(Y-T)

#To derive the IS schedule we insert the consumption, investment, and import functions to solve for $i$ from the equilibrium condition:
Y_new = Y_func.subs(C, C_func).subs(I, I_func).subs(Z, Z_func )

print(Y_new)

G + I_bar + X + a - alpha + b*(-T + Y) - beta*(-T + Y) - d*i


In [16]:
def i_LM(c_1, c_2, c_3, Ms, P, Y):
    i_LM = (c_1 - Ms/P)/c_3 + c_2/c_3*Y
    return i_LM

i_2 = i_LM(c_1, c_2, c_3, Ms, P, Y)

In [15]:
i_LM_func = sm.lambdify(args = (c_1, c_2, c_3, Ms, P, Y),
                     expr = i_2)

SyntaxError: cannot assign to literal (<lambdifygenerated-5>, line 2)

In [17]:
"2|DEFINE PARAMETERS AND ARRAYS"
# Parameters
Y_value = 100
c_1_value = 1000              # Precautionary money demand
c_2_value = 10                # Transaction money demand
c_3_value = 10                # Speculation money demand
Ms_value = 20000             # Nominal money supply
P_value  = 20                # Price level
# Arrays
Y = np.arange(Y_value)  # Array values of Y

i_LM_ana = i_LM_func(c_1_value, c_2_value, c_3_value, Ms_value, P_value, Y_value)

print(f' The analytical solution for i_LM is: {i_LM_ana:.3f}')

 The analytical solution for i_LM is: 100.000


# Numerical solution
The numerical solution is solved using an objective function (minimization problem function

In [19]:
#We define the objective function

obj_i_IS = lambda i: i-i_2

# We run and print the numerical solution function using a root optimizer.  
ss_num = optimize.root_scalar(obj_i_IS, 
                              bracket = [0.01,100], 
                              method = 'brentq')

print(f'The numerical solution for capital per worker is: {ss_num.root:.3f} which is the same value as found in the analytical solution ({kss_an:.3f})') 

ValueError: f(a) and f(b) must have different signs